In [ ]:
!pip install langchain langchain-community unstructured openpyxl pytesseract

In [ ]:
!pip install together

In [ ]:
pip install gradio pandas numpy matplotlib seaborn plotly together langchain langchain-community pytesseract pillow pdf2image pymupdf python-docx openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.9 MB/s eta 0:00:00


In [ ]:
!pip install -q together gradio pandas plotly docx2txt PyMuPDF python-pptx openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 11.0 MB/s eta 0:00:00


In [32]:
!pip install pdfplumber
!pip install pypdf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [35]:
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from PyPDF2 import PdfReader
from docx import Document
import together

# Set API Key
os.environ["TOGETHER_API_KEY"] = "tgp_v1_S8aX30kzD2NdIch1q0iO2kl_v8EgWme1DZEJLrQC6Do"
together.api_key = os.environ["TOGETHER_API_KEY"]

LLM_MODEL = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"
last_call_time = 0
cooldown_seconds = 90

# 📄 Extract Text from Any File
def extract_text(file):
    if file is None:
        return "❌ No file uploaded.", None

    ext = os.path.splitext(file.name)[-1].lower()
    try:
        if ext == ".pdf":
            pdf = PdfReader(file)
            text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
        elif ext == ".docx":
            doc = Document(file)
            text = "\n".join(p.text for p in doc.paragraphs)
        elif ext in [".txt"]:
            text = file.read().decode("utf-8")
        elif ext in [".csv", ".xlsx"]:
            df = pd.read_csv(file) if ext == ".csv" else pd.read_excel(file)
            return df.to_string(), df
        else:
            return "❌ Unsupported file type.", None
        return text, None
    except Exception as e:
        return f"⚠️ Extraction error: {e}", None

# 🧠 Query LLaMA
def query_llama(prompt):
    import time
    global last_call_time
    elapsed = time.time() - last_call_time
    if elapsed < cooldown_seconds:
        return f"⏳ Please wait {int(cooldown_seconds - elapsed)} seconds due to rate limits."

    try:
        response = together.Complete.create(
            model=LLM_MODEL,
            prompt=prompt,
            max_tokens=512,
            temperature=0.2,
            stop=["<|endoftext|>"]
        )
        last_call_time = time.time()

        # ✅ Extracting the output properly from the 'choices' list
        if 'choices' in response and len(response['choices']) > 0:
            return response['choices'][0]['text'].strip()
        else:
            return f"⚠️ Unexpected response format: {response}"

    except Exception as e:
        return f"❌ LLaMA error: {e}"


# 📊 Create Chart
def create_chart(df, x_col, y_col):
    if df is None or x_col not in df.columns or y_col not in df.columns:
        return "⚠️ Please select valid columns", None

    try:
        plt.figure(figsize=(8, 4))
        df.plot(kind="bar", x=x_col, y=y_col)
        chart_path = "/tmp/chart.png"
        plt.savefig(chart_path)
        plt.close()
        return "✅ Chart created!", chart_path
    except Exception as e:
        return f"⚠️ Chart error: {e}", None

# 🌐 Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 📄 AI Agent with LLaMA-4 - Intelligent Q&A + Visualizations")

    file = gr.File(label="Upload File", file_types=[".pdf", ".txt", ".docx", ".csv", ".xlsx"], type="filepath")
    extract_btn = gr.Button("Extract Text")

    text_output = gr.Textbox(label="Extracted Text")
    df_state = gr.State(None)
    full_text_state = gr.State("")

    with gr.Row():
        question = gr.Textbox(label="Ask LLaMA a question")
        ask_btn = gr.Button("Ask")

    answer = gr.Textbox(label="LLaMA's Answer")

    with gr.Row():
        x_col = gr.Dropdown(label="X-axis")
        y_col = gr.Dropdown(label="Y-axis")
        chart_btn = gr.Button("Create Chart")

    chart_output = gr.Image(label="Chart")

    # Events
    def handle_extraction(file):
        text, df = extract_text(file)
        cols = df.columns.tolist() if df is not None else []
        return text, df, text, gr.update(choices=cols), gr.update(choices=cols)

    extract_btn.click(handle_extraction, [file], [text_output, df_state, full_text_state, x_col, y_col])
    ask_btn.click(lambda text: query_llama(f"Document:\n{text}\n\nAnswer:"), [full_text_state], answer)
    chart_btn.click(create_chart, [df_state, x_col, y_col], [answer, chart_output])

demo.launch(debug=False)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f86a85e1427b51b1fd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
